# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 4 2023 1:15PM,254288,10,Yusin-8989598,Yusen – 3D Matrix,Released
1,Jan 4 2023 1:09PM,254287,12,U1166,"Uniderm USA, Inc",Released
2,Jan 4 2023 1:09PM,254287,12,U1167,"Uniderm USA, Inc",Released
3,Jan 4 2023 1:09PM,254287,12,U1168,"Uniderm USA, Inc",Released
4,Jan 4 2023 1:01PM,254286,19,MSP217219,"Methapharm, Inc.",Released
5,Jan 4 2023 12:55PM,254284,12,HH8661,Hush Hush,Released
6,Jan 4 2023 12:47PM,254282,19,ADV80010028,"AdvaGen Pharma, Ltd",Released
7,Jan 4 2023 12:47PM,254282,19,ADV80010029,"AdvaGen Pharma, Ltd",Released
8,Jan 4 2023 12:47PM,254282,19,ADV80010030,"AdvaGen Pharma, Ltd",Released
9,Jan 4 2023 12:47PM,254282,19,ADV80010031,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
24,254282,Released,14
25,254284,Released,1
26,254286,Released,1
27,254287,Released,3
28,254288,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
254282,NaN,NaN,14.0
254284,NaN,NaN,1.0
254286,NaN,NaN,1.0
254287,NaN,NaN,3.0
254288,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254193,0.0,1.0,0.0
254195,0.0,3.0,8.0
254196,0.0,0.0,17.0
254198,0.0,2.0,24.0
254199,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254193,0,1,0
254195,0,3,8
254196,0,0,17
254198,0,2,24
254199,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254193,0,1,0
1,254195,0,3,8
2,254196,0,0,17
3,254198,0,2,24
4,254199,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254193,,1,
1,254195,,3,8
2,254196,,,17
3,254198,,2,24
4,254199,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 4 2023 1:15PM,254288,10,Yusen – 3D Matrix
1,Jan 4 2023 1:09PM,254287,12,"Uniderm USA, Inc"
4,Jan 4 2023 1:01PM,254286,19,"Methapharm, Inc."
5,Jan 4 2023 12:55PM,254284,12,Hush Hush
6,Jan 4 2023 12:47PM,254282,19,"AdvaGen Pharma, Ltd"
20,Jan 4 2023 12:42PM,254280,10,"Methapharm, Inc."
21,Jan 4 2023 12:39PM,254278,16,Innova Softgel LLC
22,Jan 4 2023 12:39PM,254277,18,Innova Softgel LLC
23,Jan 4 2023 11:46AM,254275,19,"Avet Pharmaceuticals Labs, Inc"
24,Jan 4 2023 11:34AM,254274,10,Acute Outpatient Solutions


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 4 2023 1:15PM,254288,10,Yusen – 3D Matrix,,,1
1,Jan 4 2023 1:09PM,254287,12,"Uniderm USA, Inc",,,3
2,Jan 4 2023 1:01PM,254286,19,"Methapharm, Inc.",,,1
3,Jan 4 2023 12:55PM,254284,12,Hush Hush,,,1
4,Jan 4 2023 12:47PM,254282,19,"AdvaGen Pharma, Ltd",,,14
5,Jan 4 2023 12:42PM,254280,10,"Methapharm, Inc.",,,1
6,Jan 4 2023 12:39PM,254278,16,Innova Softgel LLC,,,1
7,Jan 4 2023 12:39PM,254277,18,Innova Softgel LLC,,,1
8,Jan 4 2023 11:46AM,254275,19,"Avet Pharmaceuticals Labs, Inc",,,1
9,Jan 4 2023 11:34AM,254274,10,Acute Outpatient Solutions,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 4 2023 1:15PM,254288,10,Yusen – 3D Matrix,1,,
1,Jan 4 2023 1:09PM,254287,12,"Uniderm USA, Inc",3,,
2,Jan 4 2023 1:01PM,254286,19,"Methapharm, Inc.",1,,
3,Jan 4 2023 12:55PM,254284,12,Hush Hush,1,,
4,Jan 4 2023 12:47PM,254282,19,"AdvaGen Pharma, Ltd",14,,
5,Jan 4 2023 12:42PM,254280,10,"Methapharm, Inc.",1,,
6,Jan 4 2023 12:39PM,254278,16,Innova Softgel LLC,1,,
7,Jan 4 2023 12:39PM,254277,18,Innova Softgel LLC,1,,
8,Jan 4 2023 11:46AM,254275,19,"Avet Pharmaceuticals Labs, Inc",1,,
9,Jan 4 2023 11:34AM,254274,10,Acute Outpatient Solutions,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 4 2023 1:15PM,254288,10,Yusen – 3D Matrix,1,,
1,Jan 4 2023 1:09PM,254287,12,"Uniderm USA, Inc",3,,
2,Jan 4 2023 1:01PM,254286,19,"Methapharm, Inc.",1,,
3,Jan 4 2023 12:55PM,254284,12,Hush Hush,1,,
4,Jan 4 2023 12:47PM,254282,19,"AdvaGen Pharma, Ltd",14,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 4 2023 1:15PM,254288,10,Yusen – 3D Matrix,1.0,NaN,NaN
1,Jan 4 2023 1:09PM,254287,12,"Uniderm USA, Inc",3.0,NaN,NaN
2,Jan 4 2023 1:01PM,254286,19,"Methapharm, Inc.",1.0,NaN,NaN
3,Jan 4 2023 12:55PM,254284,12,Hush Hush,1.0,NaN,NaN
4,Jan 4 2023 12:47PM,254282,19,"AdvaGen Pharma, Ltd",14.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 4 2023 1:15PM,254288,10,Yusen – 3D Matrix,1.0,0.0,0.0
1,Jan 4 2023 1:09PM,254287,12,"Uniderm USA, Inc",3.0,0.0,0.0
2,Jan 4 2023 1:01PM,254286,19,"Methapharm, Inc.",1.0,0.0,0.0
3,Jan 4 2023 12:55PM,254284,12,Hush Hush,1.0,0.0,0.0
4,Jan 4 2023 12:47PM,254282,19,"AdvaGen Pharma, Ltd",14.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1525480,21.0,1.0,0.0
12,508571,4.0,0.0,0.0
15,1016966,68.0,36.0,8.0
16,508543,2.0,0.0,0.0
18,254277,1.0,0.0,0.0
19,2033926,58.0,3.0,1.0
21,254266,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1525480,21.0,1.0,0.0
1,12,508571,4.0,0.0,0.0
2,15,1016966,68.0,36.0,8.0
3,16,508543,2.0,0.0,0.0
4,18,254277,1.0,0.0,0.0
5,19,2033926,58.0,3.0,1.0
6,21,254266,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,21.0,1.0,0.0
1,12,4.0,0.0,0.0
2,15,68.0,36.0,8.0
3,16,2.0,0.0,0.0
4,18,1.0,0.0,0.0
5,19,58.0,3.0,1.0
6,21,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,21.0
1,12,Released,4.0
2,15,Released,68.0
3,16,Released,2.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,21
Status,,,,,,,
Completed,0.0,0.0,8.0,0.0,0.0,1.0,0.0
Executing,1.0,0.0,36.0,0.0,0.0,3.0,0.0
Released,21.0,4.0,68.0,2.0,1.0,58.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,21
0,Completed,0.0,0.0,8.0,0.0,0.0,1.0,0.0
1,Executing,1.0,0.0,36.0,0.0,0.0,3.0,0.0
2,Released,21.0,4.0,68.0,2.0,1.0,58.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,21
0,Completed,0.0,0.0,8.0,0.0,0.0,1.0,0.0
1,Executing,1.0,0.0,36.0,0.0,0.0,3.0,0.0
2,Released,21.0,4.0,68.0,2.0,1.0,58.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()